![piggy_bank](piggy_bank.jpg)

<br>

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean and store the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to set up a PostgreSQL database to store this campaign's data, designing the schema in a way that would allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split, in order to save separate files based on the tables you will create. It is recommended to use `pandas` for these tasks.

Lastly, you will write the SQL code that the bank can execute to create the tables and populate with the data from the csv files. As the bank are quite strict about their security, you'll provide the database design script as a `.sql` file that they can then run. 

You have been asked to design a database that will have three tables:

## client

| column | data type | description |
|--------|-----------|-------------|
| `id` | `serial` | Client ID - primary key |
| `age` | `integer` | Client's age in years |
| `job` | `text` | Client's type of job |
| `marital` | `text` | Client's marital status |
| `education` | `text` | Client's level of education |
| `credit_default` | `boolean` | Whether the client's credit is in default |
| `housing` | `boolean` | Whether the client has an existing housing loan (mortgage) |
| `loan` | `boolean` | Whether the client has an existing personal loan |

<br>

## campaign

| column | data type | description |
|--------|-----------|-------------|
| `campaign_id` | `serial` | Campaign ID - primary key |
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign |
| `contact_duration` | `integer` | Last contact duration in seconds |
| `pdays` | `integer` | Number of days since contact in previous campaign (`999` = not previously contacted) |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign |
| `previous_outcome` | `boolean` | Outcome of the previous campaign |
| `campaign_outcome` | `boolean` | Outcome of the current campaign |
| `last_contact_date` | `date` | Last date the client was contacted |

<br>

## economics

| column | data type | description |
|--------|-----------|-------------|
| `client_id` | `serial` | Client ID - references `id` in the `client` table |
| `emp_var_rate` | `float` | Employment variation rate (quarterly indicator) |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three month rate (daily indicator) |
| `number_employed` | `float` | Number of employees (quarterly indicator)| 

The first stage involves importing the relevant headers from the .csv file in the pandas DataFrame. The columns have been selected using the usecols arguement to specify the columns required to segment the .csv into three dataframes.

In [225]:
import pandas as pd
import numpy as np

client_rows = ['client_id','age','job','marital','education',
                                  'credit_default','housing','loan']

df_client = pd.read_csv('bank_marketing.csv',
                       usecols = client_rows)

campaign_rows = ['client_id','contact','month','day','duration','campaign','pdays',
                'previous','poutcome','y']
df_campaign = pd.read_csv('bank_marketing.csv',
                         usecols = campaign_rows)

economics_rows = ['client_id','emp_var_rate','cons_price_idx','cons_conf_idx',
                  'euribor3m','nr_employed']
df_economics = pd.read_csv('bank_marketing.csv',
                          usecols=economics_rows)

The next step involves renaming certain columns in the dataframes.

In [226]:
df_client.rename(columns={'client_id':'id'},inplace=True)
print(df_client)

          id  age          job  marital            education credit_default  \
0          0   56    housemaid  married             basic.4y             no   
1          1   57     services  married          high.school        unknown   
2          2   37     services  married          high.school             no   
3          3   40       admin.  married             basic.6y             no   
4          4   56     services  married          high.school             no   
...      ...  ...          ...      ...                  ...            ...   
41183  41183   73      retired  married  professional.course             no   
41184  41184   46  blue-collar  married  professional.course             no   
41185  41185   56      retired  married    university.degree             no   
41186  41186   44   technician  married  professional.course             no   
41187  41187   74      retired  married  professional.course             no   

      housing loan  
0          no   no  
1        

In [227]:
df_campaign.rename(columns={'duration':'contact_duration',
                            'previous':'previous_campaign_contacts',
                           'y':'campaign_outcome','poutcome':'previous_outcome',
                           'campaign':'number_contacts'},inplace=True)
print(df_campaign)

       client_id    contact month  day  contact_duration  number_contacts  \
0              0  telephone   may   13               261                1   
1              1  telephone   may   19               149                1   
2              2  telephone   may   23               226                1   
3              3  telephone   may   27               151                1   
4              4  telephone   may    3               307                1   
...          ...        ...   ...  ...               ...              ...   
41183      41183   cellular   nov   30               334                1   
41184      41184   cellular   nov    6               383                1   
41185      41185   cellular   nov   24               189                2   
41186      41186   cellular   nov   17               442                1   
41187      41187   cellular   nov   23               239                3   

       pdays  previous_campaign_contacts previous_outcome campaign_outcome 

In [228]:
pd.set_option('display.max_columns', None)
df_economics.rename(columns={'euribor3m':'euribor_three_months',
                            'nr_employed':'number_employed'},inplace=True)
print(df_economics)

       client_id  emp_var_rate  cons_price_idx  cons_conf_idx  \
0              0           1.1          93.994          -36.4   
1              1           1.1          93.994          -36.4   
2              2           1.1          93.994          -36.4   
3              3           1.1          93.994          -36.4   
4              4           1.1          93.994          -36.4   
...          ...           ...             ...            ...   
41183      41183          -1.1          94.767          -50.8   
41184      41184          -1.1          94.767          -50.8   
41185      41185          -1.1          94.767          -50.8   
41186      41186          -1.1          94.767          -50.8   
41187      41187          -1.1          94.767          -50.8   

       euribor_three_months  number_employed  
0                     4.857           5191.0  
1                     4.857           5191.0  
2                     4.857           5191.0  
3                     4.857    

**Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values**.

In [229]:
df_client['education'] = df_client['education'].str.replace('.','-')
df_client['education'] = df_client['education'].str.replace('unknown',str(np.nan))
df_client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic-4y,no,no,no
1,1,57,services,married,high-school,unknown,no,no
2,2,37,services,married,high-school,no,yes,no
3,3,40,admin.,married,basic-6y,no,no,no
4,4,56,services,married,high-school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional-course,no,yes,no
41184,41184,46,blue-collar,married,professional-course,no,no,no
41185,41185,56,retired,married,university-degree,no,yes,no
41186,41186,44,technician,married,professional-course,no,no,no


**Remove periods from the "job" **column.****

In [230]:
df_client['job'] = df_client['job'].str.replace('.','')
df_client

,id,age,job,marital,education,credit_default,housing,loan
0,0,56,housemaid,married,basic-4y,no,no,no
1,1,57,services,married,high-school,unknown,no,no
2,2,37,services,married,high-school,no,yes,no
3,3,40,admin,married,basic-6y,no,no,no
4,4,56,services,married,high-school,no,no,yes
...,...,...,...,...,...,...,...,...
41183,41183,73,retired,married,professional-course,no,yes,no
41184,41184,46,blue-collar,married,professional-course,no,no,no
41185,41185,56,retired,married,university-degree,no,yes,no
41186,41186,44,technician,married,professional-course,no,no,no


**Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".**

In [231]:
df_campaign['previous_outcome'] = df_campaign['previous_outcome'].str.replace('nonexistent',str(np.nan))
df_campaign

,client_id,contact,month,day,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,telephone,may,13,261,1,999,0,nan,no
1,1,telephone,may,19,149,1,999,0,nan,no
2,2,telephone,may,23,226,1,999,0,nan,no
3,3,telephone,may,27,151,1,999,0,nan,no
4,4,telephone,may,3,307,1,999,0,nan,no
...,...,...,...,...,...,...,...,...,...,...
41183,41183,cellular,nov,30,334,1,999,0,nan,yes
41184,41184,cellular,nov,6,383,1,999,0,nan,no
41185,41185,cellular,nov,24,189,2,999,0,nan,no
41186,41186,cellular,nov,17,442,1,999,0,nan,yes


In [232]:
outcome_map = {
    'success' : 1,
    'failure' : 0
}

df_campaign['previous_outcome'].map(outcome_map)
df_campaign['campaign_outcome'].map(outcome_map)
df_campaign

,client_id,contact,month,day,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,telephone,may,13,261,1,999,0,nan,no
1,1,telephone,may,19,149,1,999,0,nan,no
2,2,telephone,may,23,226,1,999,0,nan,no
3,3,telephone,may,27,151,1,999,0,nan,no
4,4,telephone,may,3,307,1,999,0,nan,no
...,...,...,...,...,...,...,...,...,...,...
41183,41183,cellular,nov,30,334,1,999,0,nan,yes
41184,41184,cellular,nov,6,383,1,999,0,nan,no
41185,41185,cellular,nov,24,189,2,999,0,nan,no
41186,41186,cellular,nov,17,442,1,999,0,nan,yes


**Add a column called campaign_id in campaign, where all rows have a value of 1.**

In [233]:
df_campaign['campaign_id'] = 1
df_campaign

,client_id,contact,month,day,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id
0,0,telephone,may,13,261,1,999,0,nan,no,1
1,1,telephone,may,19,149,1,999,0,nan,no,1
2,2,telephone,may,23,226,1,999,0,nan,no,1
3,3,telephone,may,27,151,1,999,0,nan,no,1
4,4,telephone,may,3,307,1,999,0,nan,no,1
...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,cellular,nov,30,334,1,999,0,nan,yes,1
41184,41184,cellular,nov,6,383,1,999,0,nan,no,1
41185,41185,cellular,nov,24,189,2,999,0,nan,no,1
41186,41186,cellular,nov,17,442,1,999,0,nan,yes,1


**Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.**

In [234]:
df_campaign['last_contact_date'] = pd.to_datetime(str(2022) + df_campaign['month'].astype(str) +df_campaign['day'].astype(str),format='%Y%b%d')
df_campaign['last_contact_date'] = df_campaign['last_contact_date'].dt.date
df_campaign

,client_id,contact,month,day,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,telephone,may,13,261,1,999,0,nan,no,1,2022-05-13
1,1,telephone,may,19,149,1,999,0,nan,no,1,2022-05-19
2,2,telephone,may,23,226,1,999,0,nan,no,1,2022-05-23
3,3,telephone,may,27,151,1,999,0,nan,no,1,2022-05-27
4,4,telephone,may,3,307,1,999,0,nan,no,1,2022-05-03
...,...,...,...,...,...,...,...,...,...,...,...,...
41183,41183,cellular,nov,30,334,1,999,0,nan,yes,1,2022-11-30
41184,41184,cellular,nov,6,383,1,999,0,nan,no,1,2022-11-06
41185,41185,cellular,nov,24,189,2,999,0,nan,no,1,2022-11-24
41186,41186,cellular,nov,17,442,1,999,0,nan,yes,1,2022-11-17


**Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.**

In [235]:
df_campaign.drop(columns=['day','month'],inplace=True)
df_campaign

,client_id,contact,contact_duration,number_contacts,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,campaign_id,last_contact_date
0,0,telephone,261,1,999,0,nan,no,1,2022-05-13
1,1,telephone,149,1,999,0,nan,no,1,2022-05-19
2,2,telephone,226,1,999,0,nan,no,1,2022-05-23
3,3,telephone,151,1,999,0,nan,no,1,2022-05-27
4,4,telephone,307,1,999,0,nan,no,1,2022-05-03
...,...,...,...,...,...,...,...,...,...,...
41183,41183,cellular,334,1,999,0,nan,yes,1,2022-11-30
41184,41184,cellular,383,1,999,0,nan,no,1,2022-11-06
41185,41185,cellular,189,2,999,0,nan,no,1,2022-11-24
41186,41186,cellular,442,1,999,0,nan,yes,1,2022-11-17


**Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.**

In [236]:
df_client.to_csv('client.csv',index=False)
df_campaign.to_csv('campaign.csv',index=False)
df_economics.to_csv('economics.csv',index=False)

**Create a variable called client_table, containing SQL code as a string to create a table called client using values from client.csv.**

In [237]:
client_table = """
CREATE TABLE client (
    id SERIAL PRIMARY KEY,
    age INTEGER,
    job TEXT,
    marital TEXT,
    education TEXT,
    credit_default BOOLEAN,
    housing BOOLEAN,
    loan BOOLEAN
);
\copy client from 'client.csv' DELIMITER ',' CSV HEADER
"""

**Create a variable called campaign_table, containing SQL code as a string to create a table called campaign using values from campaign.csv.**

In [238]:
campaign_table = """
CREATE TABLE campaign (
    campaign_id SERIAL PRIMARY KEY,
    client_id SERIAL references client (id),
    number_contacts INT,
    contact_duration INT ,
    pdays INT,
    previous_campaign_contacts INT,
    previous_outcome BOOLEAN,
    campaign_outcome BOOLEAN,
    last_contact_date DATE
);
\copy campaign from 'campaign.csv' DELIMITER ',' CSV HEADER 
"""

**Create a variable called economics_table, containing SQL code as a string to create a table called economics using values from economics.csv.**

In [239]:
economics_table = """
CREATE TABLE economics (
    client_id SERIAL references client (id),
    emp_var_rate FLOAT,
    cons_price_idx FLOAT,
    euribor_three_months FLOAT,
    number_employed FLOAT
);
\copy economics from 'economics.csv' DELIMITER ',' CSV HEADER
"""